In [2]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from torch.nn.utils import parameters_to_vector
from torch.optim import Adam
from torch.distributions import MultivariateNormal, Normal
from preds.models import MLPS

from preds.likelihoods import GaussianLh
from preds.datasets import UCIClassificationDatasets
from preds.laplace import Laplace

In [3]:
width = 50 # as in Immer et al. 
depth = 2 # as in Immer et al. 
prior_prec = np.logspace(-2, 2, num=10)[0] # as in Immer et al. but depends on dataset
lr = 1e-3 # as in Immer et al. 
n_epochs = 10000
n_samples = 1000
train_size = 0.70 # as in Immer et al. 
lh = GaussianLh(sigma_noise=0.3)  
uci_dataset = 'glass'
root_dir = '../data/'
device = 'cpu'

Load training data

In [86]:
ds_train = UCIClassificationDatasets(train=True, data_set=uci_dataset, split_train_size=train_size, double=False, root=root_dir)
X_train, y_train = ds_train.data.to(device), ds_train.targets.to(device).unsqueeze(1)
train_loader = [(X_train, y_train)]  

Load validation data

In [87]:
ds_val = UCIClassificationDatasets(train=False,valid=True, data_set=uci_dataset, split_train_size=train_size, double=False, root=root_dir)
X_val, y_val = ds_val.data.to(device), ds_val.targets.to(device).unsqueeze(1)
val_loader = [(X_val, y_val)] 

In [59]:
X_train.shape

torch.Size([149, 9])

In [88]:
model = MLPS(X_train.shape[1], [width]*depth, 1, activation='tanh', flatten=False).to(device)
optim = Adam(model.parameters(), lr=lr)
losses = list()
for i in range(n_epochs):
    f = model(X_train)
    w = parameters_to_vector(model.parameters())
    reg = 0.5 * prior_prec * w @ w
    loss = - lh.log_likelihood(y_train, f) + reg
    loss.backward()
    optim.step()
    losses.append(loss.item())
    model.zero_grad()


In [89]:
lap = Laplace(model, float(prior_prec), lh)


def get_pred_for(x, model_type='glm', cov_type='full'):
    #### INFERENCE (Posterior approximation) ####
    lap.infer(train_loader, cov_type=cov_type, dampen_kron=model_type=='bnn')
    if model_type == 'glm':
        #### GLM PREDICTIVE ####
        mu, var = lap.predictive_samples_glm(x, n_samples=n_samples)
    elif model_type == 'bnn':
        #### BNN PREDICTIVE ####
        samples = lap.predictive_samples_bnn(x, n_samples=n_samples)
        mu = samples.mean(axis=0)
        var = samples.cov(axis=0)
    else:
        raise ValueError('unsupported model_type.')
    mu = mu.detach().cpu().squeeze()
    var = var.detach().cpu().squeeze()
    return mu, var

In [31]:
print(X_train.dtype)
print(X_val.dtype)
print(mu.dtype)

torch.float32
torch.float32
float32


In [90]:
# GLM

mu_glm, var_glm = get_pred_for(X_val, 'glm', 'full')# runs
# mu_glm_kron, var_glm_kron = get_pred_for(X_val, 'glm', 'kron') # doesn't run
#mu, var = get_pred_for(X_val, 'glm', 'diag')# doesn't run

# BNN
mu_bnn, var_bnn = get_pred_for(X_val, 'bnn', 'full')

AttributeError: 'Tensor' object has no attribute 'cov'

In [79]:
y_val.shape

torch.Size([33, 1])

In [91]:
lh_glm = Normal(mu_glm, var_glm)
print(-torch.mean(lh_glm.log_prob(y_val.squeeze(-1))))

tensor(6.7749)


In [65]:
lh_bnn = Normal(mu_bnn, var_bnn)
print(-torch.mean(lh_bnn.log_prob(y_val.squeeze(-1))))

RuntimeError: The size of tensor a (33) must match the size of tensor b (1000) at non-singleton dimension 0

Run the classification pipeline for chosen dataset

In [ ]:
! python3 ../experiments/classification.py --dataset glass --root_dir ../data

  0%|                                                    | 0/10 [00:00<?, ?it/s]

In [93]:
import os
os.getcwd()

'/Users/tamire1/Documents/GitHub/BNN-predictions/notebooks'